<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/uci_har_dataset_hier_with%20transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [2]:
%load_ext tensorboard


# Some methods and data which will be used below

In [3]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [4]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [5]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [6]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [7]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [8]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [9]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest


  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,xtest,ytrain,ytest


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

In [10]:

def confusion_matrix_display(cmTitle, y_test, y_pred):
    # confusion matrix usage to evaluate the quality of the output of a classifier on the data set.
    # https://github.com/todddangerfarr/mlnd-p5-capstone-wearables-activity-tracking/blob/master/human-activities-and-postural-transitions.ipyn
    cm = confusion_matrix(y_test, y_pred)
    # get the activities numpy array and convert to a list of Labels
    a = np.array(activity_labels2.Activity.unique())
    Labels = a.tolist()

    colormap = plt.cm.Blues
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes()
    sns.set(style='whitegrid', font_scale=1.5)
    sns.heatmap(cm,
                #fmt='.0f',
                fmt ='g',
                annot=True,
                annot_kws={"size": 10},
                cmap=colormap,
                square=True,
                linecolor='white',
                xticklabels =Labels,
                yticklabels =Labels,
                ax = ax)

    ax.set_title(cmTitle + ' ' + 'Testing Confusion Matrix ', fontsize=22)
    ax.set_xlabel('Predicted Label', fontsize=14)
    ax.set_ylabel('True Label', fontsize=14)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=35, ha="right", rotation_mode="anchor")

    fig.subplots_adjust(wspace=0, hspace=0)
    plt.show()

    normalized_confusion_matrix = np.array(cm, dtype=np.float32)/np.sum(cm)*100
    print("")
    print("Normalized Confusion matrix (normalized to % of total test data):")
    print(normalized_confusion_matrix)

    # print classification report of Precision-Recall, f1-score accuracy scores
    # The classification_report function builds a text report showing the main classification metrics.
    #  precision is the ability of the classifier not to label as positive a sample that is negative,
    # and recall is the ability of the classifier to find all the positive sample
    from sklearn.metrics import classification_report, accuracy_score
    print("classification_report")
    print(classification_report(y_test, y_pred, target_names=Labels))
    crvalue = classification_report(y_test, y_pred, target_names=Labels)
    return(crvalue)

# Data Preparation

## Loading data from files

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y
#/content/drive/MyDrive/UCI HAR Dataset
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '/content/drive/MyDrive/UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + '/content/drive/MyDrive/UCI HAR Dataset/')

    trainylabels = trainy
    testylabels = testy

    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("trainX shape: ",trainX.shape)
    print("trainy shape: ", trainy.shape)
    print("testX shape: ", testX.shape)
    print("testy shape: ", testy.shape)
    print("trainylabels shape: ", trainylabels.shape)
    print("testylabels shape: ", testylabels.shape)

    return trainX, trainy, testX, testy ,trainylabels,testylabels


In [14]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

trainX, trainy, testX, testy, trainylabels , testylabels = load_dataset()

trainX shape:  (7352, 128, 9)
trainy shape:  (7352, 6)
testX shape:  (2947, 128, 9)
testy shape:  (2947, 6)
trainylabels shape:  (7352, 1)
testylabels shape:  (2947, 1)


In [15]:
X = np.concatenate((trainX,testX))
Y= np.concatenate((trainy,testy))
YLabels = np.concatenate((trainylabels,testylabels))

In [16]:
X.shape,Y.shape,YLabels.shape

((10299, 128, 9), (10299, 6), (10299, 1))

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state=42)


In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes =6

print(timesteps)
print(input_dim)
print(len(X_train))

epochs = 100
batch_size = 16
n_hidden = 64
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model1.add(Dropout(0.5))
model1.add(LSTM(32))
model1.add(Dropout(0.5))
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

128
9
7209
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18944     
                                                                 
 dropout (Dropout)           (None, 128, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

Epoch 1/100
451/451 [==============================] - 14s 10ms/step - loss: 0.9737 - accuracy: 0.6199
Epoch 2/100
451/451 [==============================] - 6s 13ms/step - loss: 0.4463 - accuracy: 0.8639
Epoch 3/100
451/451 [==============================] - 5s 10ms/step - loss: 0.2864 - accuracy: 0.9147
Epoch 4/100
451/451 [==============================] - 5s 10ms/step - loss: 0.2278 - accuracy: 0.9261
Epoch 5/100
451/451 [==============================] - 6s 13ms/step - loss: 0.1997 - accuracy: 0.9324
Epoch 6/100
451/451 [==============================] - 5s 10ms/step - loss: 0.1741 - accuracy: 0.9372
Epoch 7/100
451/451 [==============================] - 5s 11ms/step - loss: 0.1650 - accuracy: 0.9420
Epoch 8/100
451/451 [==============================] - 6s 13ms/step - loss: 0.1571 - accuracy: 0.9444
Epoch 9/100
451/451 [==============================] - 5s 10ms/step - loss: 0.1417 - accuracy: 0.9503
Epoch 10/100
451/451 [==============================] - 6s 12ms/step - loss: 0.14

In [ ]:
score = model1.evaluate(X_test, Y_test)
print("Traditional model Accuracy: ", score[1])

97/97 [==============================] - 1s 7ms/step - loss: 0.0815 - accuracy: 0.9715
Traditional model Accuracy:  0.9715210199356079


In [17]:
X.shape,YLabels.shape

((10299, 128, 9), (10299, 1))

In [18]:
yy = YLabels
yy.shape

(10299, 1)

In [19]:
yy =yy.reshape(10299,)

In [20]:
yy

array([5, 5, 5, ..., 2, 2, 2])

In [15]:
%ls

drive/  sample_data/


**Machine Learning Algorithms**

In [21]:
X_train = pd.read_csv('sample_data/ucihar72features.csv', header=None)
X_train=X_train.iloc[1: , :]
Y_train = YLabels.reshape(10299,)

<ipython-input-21-7c02c085b066>:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('sample_data/ucihar72features.csv', header=None)


In [22]:

###########################################################################################
x1train,x1test,y1train,y1test=splitData2(X_train,Y_train,'dynamic')
x2train,x2test,y2train,y2test=splitData2(X_train,Y_train,'static')

###########################################################################################
x1train.shape,y1train.shape,x2train.shape,y2train.shape,x1test.shape,y1test.shape,x2test.shape,y2test.shape

((3270, 72),
 (3270,),
 (3938, 72),
 (3938,),
 (1402, 72),
 (1402,),
 (1689, 72),
 (1689,))

In [27]:
X_train = np.concatenate([x1train,x2train], axis=0)
X_test = np.concatenate([x1test,x2test], axis=0)

Y_train = np.concatenate([y1train,y2train], axis=0)
Y_test = np.concatenate([y1test,y2test], axis=0)

In [21]:
X_train.shape,Y_train.shape, X_test.shape, Y_test.shape

((7208, 72), (7208,), (3091, 72), (3091,))

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, accuracy_score


svc = LinearSVC(random_state=42, tol=1e-5, verbose=1, max_iter=10000)
svc.fit(X_train, Y_train)

svm_pred = svc.predict(X_test)
SVC_pred_test = accuracy_score(Y_test, svm_pred)*100
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, svm_pred)*100))

[LibLinear]Accuracy score TEST: 92.656


In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100]
scores=[]
for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(X_train, Y_train)
    rf_pred = rf.predict(X_test)
    scores.append(accuracy_score(Y_test, rf_pred))
    print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, rf_pred)*100))
    #Stacked_precision_recall_fscore_support  = confusion_matrix_display('Stacking Logistic Regression, GradientBoostingClassifier, RandomForest and ABclassifier)', y_test, y_pred)


Accuracy score TEST: 96.862
Accuracy score TEST: 97.573
Accuracy score TEST: 97.809


In [23]:
#######################################################################

from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100]
scores=[]
for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(x2train, y2train)
    rf_pred = rf.predict(x2test)
    scores.append(accuracy_score(y2test, rf_pred))
    print('Accuracy score TEST: {:.3f}'.format(accuracy_score(y2test, rf_pred)*100))
    #Stacked_precision_recall_fscore_support  = confusion_matrix_display('Stacking Logistic Regression, GradientBoostingClassifier, RandomForest and ABclassifier)', y_test, y_pred)

##########################################################################

Accuracy score TEST: 96.862
Accuracy score TEST: 97.573
Accuracy score TEST: 97.809


In [24]:
y2test,rf_pred

(array([6, 4, 6, ..., 6, 4, 6]), array([6, 4, 6, ..., 6, 4, 6]))

In [25]:
x2test.index = x2test.index.factorize()[0] + 0

In [26]:
rf_pred = rf.predict(x2test.loc[6,:].to_numpy().reshape(1,-1))  #x2test.loc[0,:].to_numpy().reshape(1,-1)
rf_pred

array([6])

**dynamic uci har**

In [28]:
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels = splitData(X,yy,'dynamic')

In [29]:
dynamicxtrain.shape,dynamicxtest.shape,dynamicytrain.shape,dynamicytest.shape,dynamiclabels.shape

((3270, 128, 9), (1402, 128, 9), (3270, 3), (1402, 3), (1402,))

**start transformer test**

In [30]:
import model

learningRate = 3e-4

optimizer = tf.keras.optimizers.Adam(learningRate)
optimizer2 = tf.keras.optimizers.Adam(learningRate)

dynamic_classifier = model.HART((128,9),3)
static_classifier = model.HART((128,9),3)

dynamic_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
static_classifier.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [42]:
optimizer0 = tf.keras.optimizers.Adam(learningRate)
general_classifier = model.HART((128,9),6)
general_classifier.compile(
    optimizer=optimizer0,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [31]:
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(X,yy,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((3938, 128, 9), (1689, 128, 9), (3938, 3), (1689, 3), (1689,))

In [32]:
from sklearn.utils import class_weight

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(dynamiclabels),
                                                 y = dynamiclabels.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(staticlabels),
                                                 y = staticlabels.ravel())
class_weights2 = {j : temp_weights[j] for j in range(len(temp_weights))}

In [33]:
np.unique(staticlabels)

array([4, 5, 6])

In [34]:
class_weights, class_weights2

({0: 0.8801004394224733, 1: 0.983859649122807, 2: 1.1801346801346801},
 {0: 1.0974658869395713, 1: 0.987719298245614, 2: 0.929042904290429})

In [26]:
X_train.shape,Y_train.shape

NameError: ignored

In [35]:
#tf.config.run_functions_eagerly(False)

checkpoint_filepath = "bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

history0 = dynamic_classifier.fit(
    x=dynamicxtrain,
    y=dynamicytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)

history1 = static_classifier.fit(
    x=staticxtrain,
    y=staticytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights2,
    callbacks=[checkpoint_callback],
)

Epoch 1/100
13/13 [==============================] - ETA: 0s - loss: 1.1202 - accuracy: 0.3529

13/13 [==============================] - 52s 89ms/step - loss: 1.1202 - accuracy: 0.3529
Epoch 2/100
13/13 [==============================] - ETA: 0s - loss: 1.0450 - accuracy: 0.4624

13/13 [==============================] - 1s 72ms/step - loss: 1.0450 - accuracy: 0.4624
Epoch 3/100
13/13 [==============================] - ETA: 0s - loss: 0.9545 - accuracy: 0.5636

13/13 [==============================] - 1s 48ms/step - loss: 0.9545 - accuracy: 0.5636
Epoch 4/100
13/13 [==============================] - ETA: 0s - loss: 0.7981 - accuracy: 0.7055

13/13 [==============================] - 1s 47ms/step - loss: 0.7981 - accuracy: 0.7055
Epoch 5/100
13/13 [==============================] - ETA: 0s - loss: 0.6718 - accuracy: 0.7936

...

16/16 [==============================] - 1s 52ms/step - loss: 0.3426 - accuracy: 0.9771
Epoch 97/100
16/16 [==============================] - ETA: 0s - loss: 0.3368 - accuracy: 0.9794

16/16 [==============================] - 1s 52ms/step - loss: 0.3368 - accuracy: 0.9794
Epoch 98/100
16/16 [==============================] - ETA: 0s - loss: 0.3330 - accuracy: 0.9817

16/16 [==============================] - 1s 52ms/step - loss: 0.3330 - accuracy: 0.9817
Epoch 99/100
16/16 [==============================] - ETA: 0s - loss: 0.3449 - accuracy: 0.9761

16/16 [==============================] - 1s 55ms/step - loss: 0.3449 - accuracy: 0.9761
Epoch 100/100
16/16 [==============================] - ETA: 0s - loss: 0.3494 - accuracy: 0.9718

16/16 [==============================] - 1s 75ms/step - loss: 0.3494 - accuracy: 0.9718


In [36]:
_, accuracy = dynamic_classifier.evaluate(dynamicxtest, dynamicytest)
print(f"Test accuracy(dynamic): {round(accuracy * 100, 2)}%")

_, accuracy = static_classifier.evaluate(staticxtest, staticytest)
print(f"Test accuracy(static): {round(accuracy * 100, 2)}%")

#traidtional 97.41% w 97.25
#dynamic 99.5-100
#static 94.43

44/44 [==============================] - 3s 15ms/step - loss: 0.2941 - accuracy: 0.9979
Test accuracy(dynamic): 99.79%
53/53 [==============================] - 2s 16ms/step - loss: 0.4160 - accuracy: 0.9355
Test accuracy(static): 93.55%


**end transformer test**

In [ ]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(dynamicxtrain[0])
input_dim = len(dynamicxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(X_train))

epochs = 100
batch_size = 16
n_hidden = 64
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model1.add(Dropout(0.5))
model1.add(LSTM(32))
model1.add(Dropout(0.5))
model1.add(Dense(n_classes, activation='softmax'))
model1.summary()

128
9
7209
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           18944     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 31,459
Trainable params: 31,459
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model1.fit(dynamicxtrain,
          dynamicytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

Epoch 1/100
205/205 [==============================] - 7s 10ms/step - loss: 0.8596 - accuracy: 0.5911
Epoch 2/100
205/205 [==============================] - 2s 10ms/step - loss: 0.3660 - accuracy: 0.8927
Epoch 3/100
205/205 [==============================] - 2s 11ms/step - loss: 0.1504 - accuracy: 0.9606
Epoch 4/100
205/205 [==============================] - 2s 10ms/step - loss: 0.0987 - accuracy: 0.9755
Epoch 5/100
205/205 [==============================] - 2s 12ms/step - loss: 0.0755 - accuracy: 0.9847
Epoch 6/100
205/205 [==============================] - 3s 13ms/step - loss: 0.0529 - accuracy: 0.9862
Epoch 7/100
205/205 [==============================] - 2s 10ms/step - loss: 0.0448 - accuracy: 0.9902
Epoch 8/100
205/205 [==============================] - 2s 10ms/step - loss: 0.0412 - accuracy: 0.9920
Epoch 9/100
205/205 [==============================] - 2s 10ms/step - loss: 0.0386 - accuracy: 0.9920
Epoch 10/100
205/205 [==============================] - 2s 10ms/step - loss: 0.024

In [ ]:
score = model1.evaluate(dynamicxtest, dynamicytest)
print("Traditional model Accuracy: ", score[1])

44/44 [==============================] - 1s 8ms/step - loss: 0.0235 - accuracy: 0.9964
Traditional model Accuracy:  0.996433675289154


**static uci har**

In [ ]:
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(X,yy,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((3938, 128, 9), (1689, 128, 9), (3938, 3), (1689, 3), (1689,))

In [47]:
# fit and evaluate a model
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

timesteps = len(staticxtrain[0])
input_dim = len(staticxtrain[0][0])
n_classes =3

print(timesteps)
print(input_dim)
print(len(staticxtrain))

epochs = 100
batch_size = 16
n_hidden = 64
model2 = Sequential()
# Configuring the parameters
model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
# Adding a dropout layer
model2.add(Dropout(0.5))
model2.add(LSTM(32))
model2.add(Dropout(0.5))
model2.add(Dense(n_classes, activation='softmax'))
model2.summary()

128
9
3938
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 64)           18944     
                                                                 
 dropout_21 (Dropout)        (None, 128, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_22 (Dropout)        (None, 32)                0         
                                                                 
 dense_42 (Dense)            (None, 3)                 99        
                                                                 
Total params: 31,459
Trainable params: 31,459
Non-trainable params: 0
_________________________________________________________________


In [48]:
model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model2.fit(staticxtrain,
          staticytrain,
          batch_size=batch_size,
          #validation_data=(staticxtest, staticytest),
          epochs=epochs,
          )

Epoch 1/100
247/247 [==============================] - 8s 11ms/step - loss: 0.4511 - accuracy: 0.8222
Epoch 2/100
247/247 [==============================] - 3s 10ms/step - loss: 0.2889 - accuracy: 0.8903
Epoch 3/100
247/247 [==============================] - 3s 10ms/step - loss: 0.2933 - accuracy: 0.8885
Epoch 4/100
247/247 [==============================] - 3s 11ms/step - loss: 0.2464 - accuracy: 0.9027
Epoch 5/100
247/247 [==============================] - 3s 14ms/step - loss: 0.2588 - accuracy: 0.8984
Epoch 6/100
247/247 [==============================] - 3s 10ms/step - loss: 0.2436 - accuracy: 0.9099
Epoch 7/100
247/247 [==============================] - 3s 10ms/step - loss: 0.2356 - accuracy: 0.9048
Epoch 8/100
247/247 [==============================] - 3s 10ms/step - loss: 0.2320 - accuracy: 0.9083
Epoch 9/100
247/247 [==============================] - 3s 11ms/step - loss: 0.2307 - accuracy: 0.9086
Epoch 10/100
247/247 [==============================] - 3s 13ms/step - loss: 0.235

In [49]:
score = model2.evaluate(staticxtest, staticytest)
print("Traditional model Accuracy: ", score[1])

53/53 [==============================] - 1s 8ms/step - loss: 0.1382 - accuracy: 0.9550
Traditional model Accuracy:  0.9550029635429382


**hierarchical uci har**

In [37]:
yexpect = np.concatenate([dynamiclabels,staticlabels], axis=0)

yexpect.shape

(3091,)

In [38]:
merge_arr = np.concatenate([dynamicxtest, staticxtest], axis=0)
merge_arr.shape

(3091, 128, 9)

In [40]:
merge_arr.shape,X_test.shape

((3091, 128, 9), (3091, 72))

In [54]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,9)
    if(rf_pred[i] in [1,2,3]):
        predict_x=dynamic_classifier.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(rf_pred[i] in [4,5,6]):
        predict_x=model2.predict(row)

        pred=statics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

Streaming output truncated to the last 5000 lines.
591
1/1 [==============================] - 0s 30ms/step
592
1/1 [==============================] - 0s 27ms/step
593
1/1 [==============================] - 0s 29ms/step
594
1/1 [==============================] - 0s 30ms/step
595
1/1 [==============================] - 0s 32ms/step
596
1/1 [==============================] - 0s 32ms/step
597
1/1 [==============================] - 0s 34ms/step
598
1/1 [==============================] - 0s 32ms/step
599
1/1 [==============================] - 0s 32ms/step
600
1/1 [==============================] - 0s 29ms/step
601
1/1 [==============================] - 0s 29ms/step
602
1/1 [==============================] - 0s 29ms/step
603
1/1 [==============================] - 0s 36ms/step
604
1/1 [==============================] - 0s 29ms/step
605
1/1 [==============================] - 0s 30ms/step
606
1/1 [==============================] - 0s 30ms/step
607
1/1 [==============================] - 0s 29ms/st

In [55]:
scores=[]
dynamiccount=0
staticcount=0
transitioncount=0
dynamics = np.array([1,2,3])
statics = np.array([4,5,6])
transitions = np.array([7,8,9,10,11,12])
for i in range(len(merge_arr)):#len(xtest)
    row = merge_arr[i].reshape(1,128,9)
    if(yexpect[i] in [1,2,3]):
        predict_x=dynamic_classifier.predict(row)

        pred=dynamics[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])

    elif(yexpect[i] in [4,5,6]):

        s = rf.predict(X_test[i].reshape(1, -1))  #x2test.loc[0,:].to_numpy().reshape(1,-1)
        scores.append(s[0])



    print(i)
print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))

1/1 [==============================] - 0s 58ms/step
0
1/1 [==============================] - 0s 53ms/step
1
1/1 [==============================] - 0s 36ms/step
2
1/1 [==============================] - 0s 34ms/step
3
1/1 [==============================] - 0s 43ms/step
4
1/1 [==============================] - 0s 41ms/step
5
1/1 [==============================] - 0s 46ms/step
6
1/1 [==============================] - 0s 37ms/step
7
1/1 [==============================] - 0s 37ms/step
8
1/1 [==============================] - 0s 40ms/step
9
1/1 [==============================] - 0s 37ms/step
10
1/1 [==============================] - 0s 39ms/step
11
1/1 [==============================] - 0s 36ms/step
12
1/1 [==============================] - 0s 35ms/step
13
1/1 [==============================] - 0s 33ms/step
14
1/1 [==============================] - 0s 43ms/step
15
1/1 [==============================] - 0s 34ms/step
16
1/1 [==============================] - 0s 37ms/step
17
1/1 [===============

In [56]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))

# LSTM only: 97.15
#optimal : 97.347
# rf_pred: 97.347
#########################
# rf-lstm: 97.347
# rf-transformer: 97
#transformer-lstm||rf-transformer-lstm : 97.54
# transformer(master)-transformer(dynamic)-rf(static) : 98:7

Accuracy score TEST: 98.738


In [51]:
merge_arr.shape, rf_pred.shape

((3091, 128, 9), (3091,))

**Generate Dataset For traditional models**

In [ ]:
X.shape

(10299, 128, 9)

In [ ]:
x= X.reshape(10299*128,9)

In [ ]:
a=[]
for i in range(0,10299):
    a=a+[i]*128

In [ ]:
len(a)

1318272

In [ ]:
x= pd.DataFrame(x)

In [ ]:
x['id'] = a

,0,1,2,3,4,5,6,7,8,id
0,1.012817,-0.123217,0.102934,0.000181,0.010767,0.055561,0.030191,0.066014,0.022859,0
1,1.022833,-0.126876,0.105687,0.010139,0.006579,0.055125,0.043711,0.042699,0.010316,0
2,1.022028,-0.124004,0.102102,0.009276,0.008929,0.048405,0.035688,0.074850,0.013250,0
3,1.017877,-0.124928,0.106553,0.005066,0.007489,0.049775,0.040402,0.057320,0.017751,0
4,1.023680,-0.125767,0.102814,0.010810,0.006141,0.043013,0.047097,0.052343,0.002553,0
...,...,...,...,...,...,...,...,...,...,...
1318267,0.908386,-0.423054,-0.092933,-0.061667,-0.175584,0.151117,0.148295,-0.015923,0.109040,10298
1318268,0.898984,-0.392272,-0.063138,-0.070890,-0.145071,0.181814,0.143136,-0.024389,0.006547,10298
1318269,0.918862,-0.351680,-0.072539,-0.050755,-0.104717,0.173271,0.095931,-0.021024,-0.051342,10298
1318270,0.949475,-0.267526,-0.050975,-0.019807,-0.020764,0.195638,0.090708,-0.041893,-0.078877,10298


In [ ]:
from tsfresh.feature_extraction import extract_features, MinimalFCParameters, EfficientFCParameters

minimalFCParametersForTsFresh = MinimalFCParameters()
extracted_features = extract_features(x,column_id="id",default_fc_parameters = minimalFCParametersForTsFresh)

Feature Extraction: 100%|██████████| 92691/92691 [00:53<00:00, 1726.01it/s]


In [ ]:
extracted_features.columns

Index(['0__median', '0__mean', '0__standard_deviation', '0__variance',
       '0__root_mean_square', '0__maximum', '0__absolute_maximum',
       '0__minimum', '1__median', '1__mean', '1__standard_deviation',
       '1__variance', '1__root_mean_square', '1__maximum',
       '1__absolute_maximum', '1__minimum', '2__median', '2__mean',
       '2__standard_deviation', '2__variance', '2__root_mean_square',
       '2__maximum', '2__absolute_maximum', '2__minimum', '3__median',
       '3__mean', '3__standard_deviation', '3__variance',
       '3__root_mean_square', '3__maximum', '3__absolute_maximum',
       '3__minimum', '4__median', '4__mean', '4__standard_deviation',
       '4__variance', '4__root_mean_square', '4__maximum',
       '4__absolute_maximum', '4__minimum', '5__median', '5__mean',
       '5__standard_deviation', '5__variance', '5__root_mean_square',
       '5__maximum', '5__absolute_maximum', '5__minimum', '6__median',
       '6__mean', '6__standard_deviation', '6__variance',
   

In [ ]:
extracted_features = extracted_features.drop(['0__length'], axis=1)
extracted_features = extracted_features.drop(['1__length'], axis=1)
extracted_features = extracted_features.drop(['2__length'], axis=1)
extracted_features = extracted_features.drop(['3__length'], axis=1)
extracted_features = extracted_features.drop(['4__length'], axis=1)
extracted_features = extracted_features.drop(['5__length'], axis=1)
extracted_features = extracted_features.drop(['0__sum_values'], axis=1)
extracted_features = extracted_features.drop(['1__sum_values'], axis=1)
extracted_features = extracted_features.drop(['2__sum_values'], axis=1)
extracted_features = extracted_features.drop(['3__sum_values'], axis=1)
extracted_features = extracted_features.drop(['4__sum_values'], axis=1)
extracted_features = extracted_features.drop(['5__sum_values'], axis=1)

In [ ]:
extracted_features = extracted_features.drop(['6__length'], axis=1)
extracted_features = extracted_features.drop(['7__length'], axis=1)
extracted_features = extracted_features.drop(['8__length'], axis=1)
extracted_features = extracted_features.drop(['6__sum_values'], axis=1)
extracted_features = extracted_features.drop(['7__sum_values'], axis=1)
extracted_features = extracted_features.drop(['8__sum_values'], axis=1)

In [ ]:
extracted_features.to_csv('ucihar72features.csv',index=False)
